<a href="https://colab.research.google.com/github/JeffersonEspinalA/Prueba/blob/main/Hito_4_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import numpy as np
import heapq as hq
import random
import math

In [54]:
def generarIntersecciones(calles, archivo, nfilas, ncol, coordenadas):
  n = len(calles)
  if n == nfilas + ncol:
    num = 0
    dicc_c = {}
    for calle in calles:
      dicc_c[calle] = num
      num += 1
    
    I = np.zeros([nfilas,ncol], int)
    I = I - 1

    with open(archivo) as f:
      for line in f:
        nodo, fila, columna, latitud, longitud = [elem for elem in line.split()]
        I[dicc_c[fila], dicc_c[columna] - nfilas] = int(nodo)
        coordenadas.append((float(latitud), float(longitud)))

  return I

In [55]:
class Perlin:
    def __init__(self):
        self.gradients = []
        self.lowerBound = 0


    def valueAt(self, t,max):
        if(t<self.lowerBound):
            print("ERROR: Input parameter out of bounds!")
            return
        while t >= len(self.gradients)-1+self.lowerBound:
            self.gradients.append(random.uniform(1, max))

        discarded = int(self.lowerBound)
        d1 = (t-t//1)
        d2 = d1-1
        a1 = self.gradients[(int)(t//1)-discarded]*d1
        a2 = self.gradients[(int)(t//1+1)-discarded]*d2

        amt = self.__ease(d1)

        return self.__lerp(a1,a2,amt)

    def discard(self, amount):
        gradientsToDiscard = int(amount+self.lowerBound%1)
        self.gradients = self.gradients[gradientsToDiscard:]
        self.lowerBound += amount

    def __ease(self, x):
        return 6*x**5-15*x**4+10*x**3


    def __lerp(self, start, stop, amt):
        return amt*(stop-start)+start

In [56]:
def updateTraffic(hour):
  noise=Perlin()
  time=[i*0.01 for i in range(1729)]
  if hour>22 or hour<4:
    values=[int(abs(noise.valueAt(i,20)*2)) for i in time]
    return values
  if hour>=4 and hour<10:
    values=[int(abs(noise.valueAt(i,50)*10)) for i in time]
    return values
  if hour>=10 and hour<14:
    values=[int(abs(noise.valueAt(i,35)*8)) for i in time]
    return values
  if hour>=14 and hour<22:
    values=[int(abs(noise.valueAt(i,45)*10)) for i in time]
    return values

In [57]:
def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [58]:
def peso2calculo(trafficValue):
  a=50-trafficValue
  if a>=25:
    peso2=1.3
    return peso2
  else:
    peso2=0.7
    return peso2

In [59]:
def generarListaAdyacencia(I, coordenadas, hora):
  G = []
  nfilas, ncol = np.shape(I) 
  values=updateTraffic(hora)
  #Agregar las adyacencias de cada nodo
  for i in range(nfilas):
    for j in range(ncol):
      if I[i,j] > -1:
        aux = []
        latitud1, longitud1 = coordenadas[I[i,j]]
        #arriba
        if (i > 0) and (I[i-1, j] > -1):
          latitud2, longitud2 = coordenadas[I[i-1, j]]
          peso1 = haversine(latitud1, longitud1, latitud2, longitud2)
          traficcValue=values[i+1]
          peso2=peso2calculo(traficcValue)
          peso=peso1*peso2
          aux.append((I[i-1, j], peso))
        #izquierda
        if (j > 0) and (I[i, j-1] > -1):
          latitud2, longitud2 = coordenadas[I[i, j-1]]
          peso1 = haversine(latitud1, longitud1, latitud2, longitud2)
          traficcValue=values[i+1]
          peso2=peso2=peso2calculo(traficcValue)
          peso=peso1*peso2
          aux.append((I[i, j-1], peso))
        #derecha (bug)
        if (j < ncol-1) and (I[i, j+1] > -1):
          latitud2, longitud2 = coordenadas[I[i, j+1]]
          peso1 = haversine(latitud1, longitud1, latitud2, longitud2)
          traficcValue=values[i+1]
          peso2=peso2=peso2calculo(traficcValue)
          peso=peso1*peso2
          aux.append((I[i, j+1], peso))
        #abajo (bug)
        if (i < nfilas-1) and (I[i+1, j] > -1):
          latitud2, longitud2 = coordenadas[I[i+1, j]]
          peso1 = haversine(latitud1, longitud1, latitud2, longitud2)
          traficcValue=values[i+1]
          peso2=peso2calculo(traficcValue)
          peso=peso1*peso2
          aux.append((I[i+1, j], peso))
        G.append(aux)

  #Guardar en un archivo
  with open("Lista_Adyacencia_Generado.txt", 'w') as archivo:
    for lista in G:
      for num in lista:
        archivo.write(str(num) + " ")
      archivo.write("\n")

  return G

#Inputs

In [60]:
calles = []
coordenadas = []

with open("Calles.txt") as f:
  for line in f:
    calle = line.split()
    calles.append(*calle)

I = generarIntersecciones(calles, "Intersecciones con coordenadas.txt", 42, 44, coordenadas)
print(coordenadas)
print()
a = generarListaAdyacencia(I, coordenadas,9)
print(a)

[(-34.885116, -57.976523), (-34.887932, -57.974166), (-34.89187, -57.967053), (-34.892609, -57.966013), (-34.893386, -57.964961), (-34.894156, -57.963899), (-34.894975, -57.962831), (-34.895855, -57.961672), (-34.8967, -57.960551), (-34.897535, -57.959387), (-34.898376, -57.958293), (-34.899199, -57.957182), (-34.900923, -57.954865), (-34.901799, -57.953728), (-34.902535, -57.95256), (-34.90341, -57.951425), (-34.904207, -57.950321), (-34.905021, -57.949226), (-34.905919, -57.947912), (-34.906706, -57.946834), (-34.907454, -57.945938), (-34.908123, -57.945123), (-34.908721, -57.944345), (-34.909192, -57.943578), (-34.909698, -57.942918), (-34.910582, -57.941663), (-34.911149, -57.941008), (-34.911673, -57.940308), (-34.912278, -57.939462), (-34.912903, -57.938577), (-34.913625, -57.937654), (-34.914425, -57.936533), (-34.915252, -57.935428), (-34.916151, -57.934237), (-34.916991, -57.933132), (-34.917831, -57.932005), (-34.918645, -57.930889), (-34.919467, -57.929838), (-34.88653, -57.

In [61]:
def dijkstra(G, s, e, p):
  n = len(G)
  visited = [False]*n
  path = [-1]*n
  cost = [math.inf]*n

  cost[s] = 0
  pqueue = [(0, s)]
  while pqueue:
    g, u = hq.heappop(pqueue)
    if u == e:
      break
    if not visited[u]:
      visited[u] = True
      for v, w in G[u]:
        if not visited[v]:
          f = g + w
          if f < cost[v]:
            if (v == e) and (f <= p):
              pass
            else:
              cost[v] = f
              path[v] = u
              hq.heappush(pqueue, (f, v))
            
  return path, cost

In [62]:
def path_visualization(path, end):
  n = len(path)
  new_path = [-1]*n
  while path[end] > -1:
    new_path[end] = path[end]
    end = path[end]
  return new_path

In [63]:
def function_path(G, first, end, p):
  sp = []
  path, cost = dijkstra(G, first, end, p)
  p = cost[end]
  if p == math.inf:
    return
  else:
    n_path = path_visualization(path, end)
    return n_path, p

#Input

In [64]:
op1 = 1
op2 = 38

if (op1 >= 0 and op1 <= 1728) and (op2 >= 0 and op2 <= 1728):
  shortest_paths = []

  p = 0
  for _ in range(3):
    n_path, p = function_path(a, op1, op2, p)
    shortest_paths.append(n_path)

print(shortest_paths[0])
print(shortest_paths[1])
print(shortest_paths[2])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 39, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 